In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/TFT_energy.csv')

In [4]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,120.0,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'time'], infer_datetime_format=True)

In [6]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0,2015-01-01 00:00:00+00:00
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0,2015-01-01 01:00:00+00:00
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0,2015-01-01 02:00:00+00:00
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0,2015-01-01 03:00:00+00:00
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0,2015-01-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [7]:
df.columns

Index(['time', 'price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump',
       'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(28050, 28)

In [10]:
test_df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
28050,2018-03-14 18:00:00+00:00,50.95,37.61,1068.0,5319.0,35044.0,0.0,345.0,102.0,1006.0,...,40.0,1849.0,220.0,1.0,1.0,8.0,14265.0,6.0,761.0,2018-03-14 18:00:00+00:00
28051,2018-03-14 19:00:00+00:00,59.73,48.21,1126.0,5292.0,36230.0,0.0,351.0,101.0,1005.0,...,40.0,1821.0,220.0,4.0,2.0,8.0,13306.0,5.0,149.0,2018-03-14 19:00:00+00:00
28052,2018-03-14 20:00:00+00:00,50.89,41.11,1054.0,4318.0,35065.0,0.0,351.0,103.0,1005.0,...,40.0,1853.0,220.0,1.0,2.0,5.0,12970.0,7.0,168.0,2018-03-14 20:00:00+00:00
28053,2018-03-14 21:00:00+00:00,45.94,36.72,963.0,4047.0,32328.0,0.0,350.0,104.0,1004.0,...,20.0,1853.0,210.0,1.0,1.0,1.0,13090.0,3.0,811.0,2018-03-14 21:00:00+00:00
28054,2018-03-14 22:00:00+00:00,30.59,21.20,853.0,3726.0,29528.0,0.0,329.0,95.0,1004.0,...,0.0,1826.0,220.0,0.0,8.0,5.0,13408.0,6.0,2007.0,2018-03-14 22:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [11]:
train_ds = PandasDataset(
    train_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [12]:
test_ds = PandasDataset(
    test_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


#### training

In [13]:
K = 24
n_hours = 24*3

In [14]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=20,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-08)

estimator = DeepAREstimator(
    prediction_length=K,
    freq="H",
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    batch_size=64,
    trainer=trainer,
    dropout_rate=0.4
)

model = estimator.train(train_ds)

 66%|██████████████████████████████████████████████████████████████████████████████████████████▍                                              | 66/100 [03:00<01:33,  2.74s/it, epoch=19/200, avg_epoch_loss=3.48]
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/gluonts/mx/trainer/_base.py:474: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(


#### predicted y

In [17]:
quantiles = ['0.025', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.975']
q50 = 5

In [18]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].samples
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[58.47999954 56.72499847 56.28499985 56.45999908 56.70999908 55.33000183
 48.45500183 44.75       43.72999954 43.98500061 43.65499878 45.97999954
 45.75       45.63000107 49.90999985 49.36999893 50.58000183 54.84500122
 53.04499817 50.59000015 48.50500107 46.76499939 48.40499878 49.19499969]
(24,)
------
1


/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


[58.47999954 56.72499847 56.28499985 56.45999908 56.70999908 55.33000183
 48.45500183 44.75       43.72999954 43.98500061 43.65499878 45.97999954
 45.75       45.63000107 49.90999985 49.36999893 50.58000183 54.84500122
 53.04499817 50.59000015 48.50500107 46.76499939 48.40499878 49.19499969
 56.82500076 57.79000092 56.52000046 56.08000183 55.75500107 47.68500137
 45.52999878 44.51499939 43.27000046 44.81000137 46.06499863 45.83000183
 46.28499985 48.93000031 49.93999863 51.54499817 56.08000183 54.27999878
 50.15999985 48.56000137 48.86000061 49.         48.22499847 50.27999878]
(48,)
------
2
[58.47999954 56.72499847 56.28499985 56.45999908 56.70999908 55.33000183
 48.45500183 44.75       43.72999954 43.98500061 43.65499878 45.97999954
 45.75       45.63000107 49.90999985 49.36999893 50.58000183 54.84500122
 53.04499817 50.59000015 48.50500107 46.76499939 48.40499878 49.19499969
 56.82500076 57.79000092 56.52000046 56.08000183 55.75500107 47.68500137
 45.52999878 44.51499939 43.2700004

/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


[58.47999954 56.72499847 56.28499985 56.45999908 56.70999908 55.33000183
 48.45500183 44.75       43.72999954 43.98500061 43.65499878 45.97999954
 45.75       45.63000107 49.90999985 49.36999893 50.58000183 54.84500122
 53.04499817 50.59000015 48.50500107 46.76499939 48.40499878 49.19499969
 56.82500076 57.79000092 56.52000046 56.08000183 55.75500107 47.68500137
 45.52999878 44.51499939 43.27000046 44.81000137 46.06499863 45.83000183
 46.28499985 48.93000031 49.93999863 51.54499817 56.08000183 54.27999878
 50.15999985 48.56000137 48.86000061 49.         48.22499847 50.27999878
 59.00500107 58.21500015 57.02000046 56.98500061 49.07500076 47.13999939
 45.58000183 44.49499893 45.58000183 46.27500153 46.72999954 46.95999908
 50.88499832 50.94499969 52.75       56.69499969 55.11500168 51.13000107
 49.57500076 48.38000107 48.26499939 49.58000183 51.32500076 60.43000031
 58.81499863 58.31999969 57.58499908 50.02000046 47.34000015 46.63000107
 45.65499878 46.72499847 47.68000031 47.20999908 48

...

/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


[58.47999954 56.72499847 56.28499985 ... 67.84999847 68.00499725
 67.58499908]
(165960,)
------
6915
[58.47999954 56.72499847 56.28499985 ... 66.62999725 66.51000214
 67.09999847]
(165984,)
------
6916


/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


[58.47999954 56.72499847 56.28499985 ... 67.56500244 67.40499878
 67.20999908]
(166008,)
------
6917
[58.47999954 56.72499847 56.28499985 ... 67.51499939 67.12000275
 68.01499939]
(166032,)
------


/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [19]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(6918, 24)

In [20]:
test_ds_predict_q50_supervised[0:5]

array([[58.47999954, 56.72499847, 56.28499985, 56.45999908, 56.70999908,
        55.33000183, 48.45500183, 44.75      , 43.72999954, 43.98500061,
        43.65499878, 45.97999954, 45.75      , 45.63000107, 49.90999985,
        49.36999893, 50.58000183, 54.84500122, 53.04499817, 50.59000015,
        48.50500107, 46.76499939, 48.40499878, 49.19499969],
       [56.82500076, 57.79000092, 56.52000046, 56.08000183, 55.75500107,
        47.68500137, 45.52999878, 44.51499939, 43.27000046, 44.81000137,
        46.06499863, 45.83000183, 46.28499985, 48.93000031, 49.93999863,
        51.54499817, 56.08000183, 54.27999878, 50.15999985, 48.56000137,
        48.86000061, 49.        , 48.22499847, 50.27999878],
       [59.00500107, 58.21500015, 57.02000046, 56.98500061, 49.07500076,
        47.13999939, 45.58000183, 44.49499893, 45.58000183, 46.27500153,
        46.72999954, 46.95999908, 50.88499832, 50.94499969, 52.75      ,
        56.69499969, 55.11500168, 51.13000107, 49.57500076, 48.38000107,
  

#### actual y

In [24]:
from helper import series_to_supervised, stage_series_to_supervised

In [27]:
test_df_supervised = series_to_supervised(test_df[['price']], n_hours, K)

In [28]:
test_df_supervised.iloc[:, -K:].shape

(6918, 24)

In [29]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [30]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+14),var1(t+15),var1(t+16),var1(t+17),var1(t+18),var1(t+19),var1(t+20),var1(t+21),var1(t+22),var1(t+23)
28122,63.57,73.69,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,...,46.77,46.58,46.62,46.13,45.39,45.04,41.33,29.18,36.40,47.22
28123,73.69,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,...,46.58,46.62,46.13,45.39,45.04,41.33,29.18,36.40,47.22,53.73
28124,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,...,46.62,46.13,45.39,45.04,41.33,29.18,36.40,47.22,53.73,58.17
28125,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,45.66,...,46.13,45.39,45.04,41.33,29.18,36.40,47.22,53.73,58.17,58.79
28126,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,45.66,48.20,...,45.39,45.04,41.33,29.18,36.40,47.22,53.73,58.17,58.79,55.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,73.35,73.64,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,...,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02
35036,73.64,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,...,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16
35037,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,60.70,...,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30
35038,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,60.70,66.64,...,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30,69.89


#### performance on test set

In [33]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 5.23
RMSE = 6.994
